In [4]:
import pycurl
import json
from io import BytesIO

In [ ]:
def get_metadata(url, filter):   
    # Initialize pycurl and set options
    buffer = BytesIO()
    c = pycurl.Curl()
    c.setopt(c.URL, f'{url}package_search?q=title:{filter}')
    c.setopt(c.WRITEDATA, buffer)

    # Perform the request
    c.perform()
    c.close()

    # Decode the JSON response and handle potential issues
    response = buffer.getvalue().decode('utf-8')

    # Check if the response is empty or not JSON
    if not response.strip():
        print("The response is empty.")
    else:
        try:
            data = json.loads(response)
            # Check if the API request was successful
            if data.get('success'):
                return data
            else:
                print("API did not return a successful response:", data)
        except json.JSONDecodeError as e:
            print("Failed to decode JSON:", e)
            print("Response content:", response)

def save_json(data, filename):
    with open(filename, 'w') as json_file:
        json.dump(data, json_file, indent=4)

def filter_result_by_year(data, year):
    for result in data['result']['results']:
        if str(year) in result['name']:
            return result
        
def get_resource_url_for_year(data, year):

    combined_data = {
        'data': [],
        'metadata': []
    }

    result = filter_result_by_year(data, year)

    for resource in result['resources']:
        url = resource['url']
        if "Haltepunkt" in url or "Haltestelle" in url:
            combined_data['metadata'].append(url)
        else:
            combined_data['data'].append(url)

    return combined_data


In [40]:
url = "https://ckan.opendata.swiss/api/3/action/"
filter = "Fahrzeiten"

download_url = get_resource_url_for_year(get_metadata(url, filter), 2020)
